In [38]:
from api.db import stat_db, crawl_db, CrawlCollections, StatCollections, relation_db, RelationCollections, DESCENDING, ASCENDING
import json
import time
import math
import re
import pandas as pd
import numpy as np
from datetime import datetime

In [32]:
bitcoin = crawl_db[CrawlCollections.COINMARKET_HISTORY_PRICE].find_one({'_id': 'bitcoin-cash'})

In [19]:
data = json.loads(bitcoin['data'])

In [20]:
market_cap = data['market_cap_by_available_supply']

In [29]:
for item in market_cap:
    original_date = datetime.fromtimestamp(item[0] / 1000)
    str_time = original_date.strftime('%Y-%m-%d')
    format_date = datetime.strptime(str_time, '%Y-%m-%d').timestamp()
    price_data = stat_db['history_currency_price'].find_one({'_id': format_date})
    if price_data is not None:
        stat_db['history_currency_price'].update_one({'_id': price_data['_id']}, {'$set': {'market_cap.{0}'.format('bitcoin-cash'): item[1]}})
    else:
        stat_db['history_currency_price'].insert_one({'_id': format_date, 'market_cap': {'{0}'.format('bitcoin-cash'): item[1]}})

In [3]:

coinmarket_coins_cursor = crawl_db[CrawlCollections.COINMARKET_CURRENCIES].find()
df = pd.DataFrame([( coins['data']['sort'], coins['_id'], coins['data']['name'], coins['data']['symbol'], coins['data']['url']) for coins in coinmarket_coins_cursor], columns=['sort', 'id', 'name', 'symbol', 'url'])                          
df.url = 'https://coinmarketcap.com' + df.url
writer = pd.ExcelWriter('E:/coinmarket_currency.xlsx')
df = df.sort_values(by='sort')
df.to_excel(writer, sheet_name='Sheet1', index=False)
writer.save()


In [ ]:
# 最近一次分析的日期有多少个货币
def how_many_coins_in_latest_stat_date():
    currency_history_cursor = stat_db[StatCollections.HISTORY_CURRENCY_PRICE_STAT].find().sort([('_id', DESCENDING)]).limit(1)
    latest_time_stat = currency_history_cursor[1]
    price_usd = latest_time_stat['price_usd']
    coins = list(price_usd.keys())
    print(len(coins))

## 分析某个货币的排名随时间变化数据


In [59]:
try:
    d = stat_db['history_currency_rank'].find_one({'currency_id': 'bitcoin'})
    latest = max(d['rank'], key=lambda x: x[0])
except TypeError as e:
    latest = 0
history_data = stat_db[StatCollections.HISTORY_CURRENCY_PRICE_STAT].find({'_id': {'$gt': latest}}).sort([('_id', ASCENDING)])
count = len(list(history_data))
for index,data in enumerate(history_data):
    if index / 100 == 0:
        print(str(index) + '/' + str(count))
    t = data['_id'] 
    prices = data['price_usd']
    sl = sorted([[k, prices[k]] for k in list(prices.keys())], key=lambda x: x[1], reverse=True)
    for index, item in enumerate(sl):
        stat_db['history_currency_rank'].find_one_and_update(
            {'currency_id': item[0]},
            {'$push': {'rank': [t, index, len(sl)]}},
            upsert=True)    